In [145]:
import pandas as pd
import numpy as np
import altair as alt

In [113]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [123]:
gt_df = pd.read_csv("av_mac_gt_df.csv")
model_df = pd.read_csv("av_mac_model_df.csv")

In [124]:
gt_df.Execution = 0

In [125]:
def df_astype(df):
    df['Gene'] = df['Gene'].astype(str)
    df.Rank = df.Rank.astype(int)
    df.Execution = df.Execution.astype(int)
    df['Cell Type'] = df['Cell Type'].astype(str)
    df['Rank Type'] = df['Rank Type'].astype(str)
    return df

In [126]:
#gt_df = df_astype(gt_df)
#model_df = df_astype(model_df)

In [127]:
gt_df['Gene'] = gt_df['Gene'].astype(str)
model_df['Gene'] = model_df['Gene'].astype(str)

In [128]:
gt_df = gt_df.set_index('Gene')

In [129]:
model_df.head()

,Gene,Rank,Execution,Cell Type,Rank Type
0,FTL,0,0,Alveolar Macrophage,Model Prediction
1,FTH1,1,0,Alveolar Macrophage,Model Prediction
2,IFI30,2,0,Alveolar Macrophage,Model Prediction
3,CTSD,3,0,Alveolar Macrophage,Model Prediction
4,CXCL3,4,0,Alveolar Macrophage,Model Prediction


In [130]:
df = model_df.join(gt_df, on='Gene', lsuffix='_Model')

In [131]:
df

,Gene,Rank_Model,Execution_Model,Cell Type_Model,Rank Type_Model,Rank,Execution,Cell Type,Rank Type
0,FTL,0,0,Alveolar Macrophage,Model Prediction,0,0,Alveolar Macrophage,Ground Truth
1,FTH1,1,0,Alveolar Macrophage,Model Prediction,1,0,Alveolar Macrophage,Ground Truth
2,IFI30,2,0,Alveolar Macrophage,Model Prediction,12,0,Alveolar Macrophage,Ground Truth
3,CTSD,3,0,Alveolar Macrophage,Model Prediction,14,0,Alveolar Macrophage,Ground Truth
4,CXCL3,4,0,Alveolar Macrophage,Model Prediction,34,0,Alveolar Macrophage,Ground Truth
...,...,...,...,...,...,...,...,...,...
2118,EIF4A1,47,49,Alveolar Macrophage,Model Prediction,44,0,Alveolar Macrophage,Ground Truth
2119,CXCL16,48,49,Alveolar Macrophage,Model Prediction,71,0,Alveolar Macrophage,Ground Truth
2120,FBP1,49,49,Alveolar Macrophage,Model Prediction,27,0,Alveolar Macrophage,Ground Truth
2121,GSTO1,50,49,Alveolar Macrophage,Model Prediction,59,0,Alveolar Macrophage,Ground Truth


In [197]:
sorted_genes = gt_df.index.values.tolist()

In [215]:
chart = alt.Chart(df).mark_circle(size=50).encode(
    y=alt.Y("Rank_Model:Q", axis=alt.Axis(title="Model Rank")),
    x=alt.X("Rank:Q", axis=alt.Axis(title="Ground Truth Rank")),
    color=alt.X("Execution_Model:N")
)
joint_chart = chart + chart.transform_regression('Rank', 'Rank_Model').mark_line(stroke='black')

joint_chart = joint_chart.properties(
    title='Alveolar Macrophage Rank Correlation Plot',
    width=900,
    height=600
)
joint_chart

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t

alt.LayerChart(...)

In [216]:
joint_chart.save("plot_av_mac_rank_corr_scatter.png")
joint_chart.save("plot_av_mac_rank_corr_scatter.pdf")

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t

In [217]:
df['Model Rank'] = df['Rank_Model']
chart = alt.Chart(df).mark_boxplot(size=6).encode(
    y=alt.Y("Model Rank:Q"),
    x=alt.X("Gene:N", axis=alt.Axis(title="Ground Truth Rank"), scale=alt.Scale(domain=sorted_genes)),
)

chart = chart.properties(
    title='Alveolar Macrophage Rank Correlation Box Plots',
    width=1000,
    height=600
)
chart

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t

alt.Chart(...)

In [218]:
chart.save("plot_av_mac_rank_corr_box.png")
chart.save("plot_av_mac_rank_corr_box.pdf")

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t

In [182]:
std_df = df.groupby(by='Gene')[['Model Rank']].std()
std_df = std_df.rename(columns={'Model Rank': 'Model Rank Std Dev'})

In [183]:
std_df

,Model Rank Std Dev
Gene,
ACP5,7.085196
ALOX5,8.895223
ALOX5AP,11.318059
ANXA5,10.170149
APOC1,6.039793
...,...
TFRC,8.504901
TREM1,7.503502
TYROBP,11.001894


In [184]:
df = df.join(std_df, on='Gene')

In [223]:
chart = alt.Chart(df).mark_circle(size=50).encode(
    y=alt.Y("Model Rank Std Dev:Q"),
    x=alt.X("Gene:O", axis=alt.Axis(title="Ground Truth Rank"), scale=alt.Scale(domain=sorted_genes)),
)

chart = chart.properties(
    title='Alveolar Macrophage Standard Deviation',
    width=1000,
    height=600
)
chart

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t

alt.Chart(...)

In [224]:
chart.save("plot_av_mac_rank_std_dev.png")
chart.save("plot_av_mac_rank_std_dev.pdf")

/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/mkeller/software/miniconda3/envs/hd-vis/lib/python3.11/site-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(t